# Amazon Forecastで利用するデータを作成していく
Forecastはヘッダなしなので注意

## 期間
target:2014/1/1-2014/6/30  
related:2014/1/1-2014/7/2  -> 時間(hour:0-23)、曜日(0:月曜 - 6:日曜, or 1-hot)  
item:期間なし -> 時系列クラスタの番号

## 0.環境準備
ライフサイクルコンフィグで環境変数を設定すれば不要

In [24]:
import sys
print(sys.path)
sys.path.append('/usr/local/lib/python3.7/site-packages')
print('-------------------------------')
print(sys.path)

['/root/data/study-timeseries-analysis/AmazonForecast_vs_XGBoost/notebook', '/opt/.sagemakerinternal/conda/lib/python37.zip', '/opt/.sagemakerinternal/conda/lib/python3.7', '/opt/.sagemakerinternal/conda/lib/python3.7/lib-dynload', '', '/opt/.sagemakerinternal/conda/lib/python3.7/site-packages', '/opt/.sagemakerinternal/conda/lib/python3.7/site-packages/IPython/extensions', '/root/.ipython', '/usr/local/lib/python3.7/site-packages']
-------------------------------
['/root/data/study-timeseries-analysis/AmazonForecast_vs_XGBoost/notebook', '/opt/.sagemakerinternal/conda/lib/python37.zip', '/opt/.sagemakerinternal/conda/lib/python3.7', '/opt/.sagemakerinternal/conda/lib/python3.7/lib-dynload', '', '/opt/.sagemakerinternal/conda/lib/python3.7/site-packages', '/opt/.sagemakerinternal/conda/lib/python3.7/site-packages/IPython/extensions', '/root/.ipython', '/usr/local/lib/python3.7/site-packages', '/usr/local/lib/python3.7/site-packages']


## 1.データセット読み込み

In [25]:
import pandas as pd

In [26]:
#df = pd.read_csv('./electricityusage_20140101_20140630.csv', names=['date','demand','item_id'])
df = pd.read_csv('../input/electricityusagedata.zip', names=['date','demand','item_id'])

In [27]:
print(df.shape)
print(df['date'].min())
print(df['date'].max())
df.head()

(3241200, 3)
2014-01-01 01:00:00
2015-01-01 00:00:00


,date,demand,item_id
0,2014-01-01 01:00:00,2.538071,client_0
1,2014-01-01 01:00:00,23.648649,client_1
2,2014-01-01 01:00:00,0.000000,client_2
3,2014-01-01 01:00:00,144.817073,client_3
4,2014-01-01 01:00:00,75.000000,client_4


## 2.特徴量生成

### 1)時刻型に変換し、特徴量（時間帯、曜日）を抽出

2014/1/1はWednesday(2)

In [28]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')

In [29]:
df['num_of_week'] = df['date'].dt.dayofweek

In [30]:
df['hour'] = df['date'].dt.hour

In [31]:
df.head()

,date,demand,item_id,num_of_week,hour
0,2014-01-01 01:00:00,2.538071,client_0,2,1
1,2014-01-01 01:00:00,23.648649,client_1,2,1
2,2014-01-01 01:00:00,0.000000,client_2,2,1
3,2014-01-01 01:00:00,144.817073,client_3,2,1
4,2014-01-01 01:00:00,75.000000,client_4,2,1


## target

In [32]:
df_tgt = df[df.date<'2014-07-01 00:00:00'][['date','demand','item_id']]

In [33]:
print(df_tgt.shape)
df_tgt.tail()

(1606910, 3)


,date,demand,item_id
1606905,2014-06-30 23:00:00,22.235225,client_365
1606906,2014-06-30 23:00:00,794.117647,client_366
1606907,2014-06-30 23:00:00,192.404007,client_367
1606908,2014-06-30 23:00:00,839.809384,client_368
1606909,2014-06-30 23:00:00,23013.513514,client_369


In [34]:
df_tgt.to_csv('../input_for_forecast/tgt_20140101_20140630.csv', index=False, header=False)

## 関連時系列データ
TODO:曜日を 1-hot-encoding

In [35]:
df_rlt = df[df.date<'2014-07-03 00:00:00'][['date','item_id','num_of_week','hour']]

In [36]:
print(df_rlt.shape)
df_rlt.tail()

(1624670, 4)


,date,item_id,num_of_week,hour
1624665,2014-07-02 23:00:00,client_365,2,23
1624666,2014-07-02 23:00:00,client_366,2,23
1624667,2014-07-02 23:00:00,client_367,2,23
1624668,2014-07-02 23:00:00,client_368,2,23
1624669,2014-07-02 23:00:00,client_369,2,23


In [37]:
df_rlt.to_csv('../input_for_forecast/rlt_20140101_20140702.csv', index=False, header=False)

## 追加実験：関連時系列にdateとitem_idのみ持たせる

In [39]:
df_rlt[['date', 'item_id']].to_csv('../input_for_forecast/rlt_time_itemid_20140101_20140702.csv', index=False, header=False)

## 1-hot 関連時系列データ

In [16]:
df_rlt_hour1hot = pd.get_dummies(df_rlt, columns=['num_of_week'])

In [18]:
print(df_rlt_hour1hot.shape)
df_rlt_hour1hot.tail()

(1624670, 10)


,date,item_id,hour,num_of_week_0,num_of_week_1,num_of_week_2,num_of_week_3,num_of_week_4,num_of_week_5,num_of_week_6
1624665,2014-07-02 23:00:00,client_365,23,0,0,1,0,0,0,0
1624666,2014-07-02 23:00:00,client_366,23,0,0,1,0,0,0,0
1624667,2014-07-02 23:00:00,client_367,23,0,0,1,0,0,0,0
1624668,2014-07-02 23:00:00,client_368,23,0,0,1,0,0,0,0
1624669,2014-07-02 23:00:00,client_369,23,0,0,1,0,0,0,0


In [17]:
df_rlt_hour1hot.to_csv('../input_for_forecast/rlt_hour1hot_20140101_20140702.csv', index=False, header=False)

## アイテムメタデータ

In [18]:
df_item = pd.read_csv('../tsl/data/lbl.csv')

In [20]:
print(df_item.shape)
df_item.head()

(370, 2)


,item_id,label
0,client_0,1
1,client_1,0
2,client_10,1
3,client_100,0
4,client_101,1


In [21]:
df_item.to_csv('../input_for_forecast/imeta_ts_cluster_3.csv', index=False, header=False)

# ファイルをS3にアップロード

## 終了